In [1]:
import pandas as pd

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from string import punctuation

sw_indo = stopwords.words("indonesian") + list(punctuation)

# Import Data

In [3]:
df = pd.read_csv("data/kompas.csv")
df.head()

,teks
0,Ginandjar Tetap Ditahan. Jaksa Agung Dilaporka...
1,Jakarta Dikangkangi Para Preman\nKALAU tak pun...
2,Penyimpangan di Setpres Seolah Terjadi Sekaran...
3,"Dibayarkan, Rapel Kenaikan Gaji Pegawai Pos\nK..."
4,"Stop Kekerasan, Elite agar Duduk Bersama\nSeju..."


# Extract BoW

In [5]:
from sklearn.feature_extraction.text import CountVectorizer # encoding dengan cara countVector

In [9]:
bow = CountVectorizer(ngram_range=(1, 2), tokenizer=word_tokenize, stop_words=sw_indo, min_df=5) # gakharus sw indo, sebaiknya list manual
bow_matrix = bow.fit_transform(df.teks)

E:\anaconda3\envs\jcopdl\lib\site-packages\sklearn\feature_extraction\text.py:396: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['``'] not in stop_words.
  warnings.warn(


# Topic Modeling

In [10]:
vocab = bow.get_feature_names() #seluruh vocabulary

E:\anaconda3\envs\jcopdl\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


## Latent Semantic Analysis (LSA) 

In [12]:
len(vocab)

25134

In [13]:
from sklearn.decomposition import TruncatedSVD

In [15]:
lsa = TruncatedSVD(n_components=10, n_iter=10, random_state=42) # n_components jumlah topik
lsa_matrix = lsa.fit_transform(bow_matrix) # bow matrix di reduce oleh lsa

keluaran setelah dekomposisi adalah weight / code 

In [17]:
print(bow_matrix.shape) # hidden 
print(lsa_matrix.shape) # weight / code
print(lsa.components_.shape) # fitur / topic

(2008, 25134)
(2008, 10)
(10, 25134)


In [19]:
def get_topic(model):
    return [[vocab[idx] for idx in reversed(comp.argsort()[-6:]) if vocab[idx].isalnum()]
           for comp in model.components_] # komponen di looping 

catatan

- for comp in model.components_ # komponen di looping

- comp.argsort()[-6:] # komponen value yang paling besar asumsi kata2 atau vocab yang penting
- [vocab[idx]  ditranslate jadi tulisan / baca vocab 

In [21]:
get_topic(lsa)

[['presiden', 'indonesia', 'pemerintah', 'dpr'],
 ['presiden', 'dpr', 'ketua', 'partai', 'mpr', 'tandjung'],
 ['pemerintah', 'rp', 'indonesia', 'bank', 'persen', 'utang'],
 ['rp', 'tandjung', 'dana', 'bulog', 'hukum', 'harga'],
 ['presiden', 'air', 'banjir', 'harga', 'rp', 'dpr'],
 ['harga', 'beras', 'rp', 'bbm'],
 ['mpr', 'konstitusi', 'bppn', 'uud'],
 ['indonesia', 'mpr', 'konstitusi', 'uud', 'perubahan', '1945'],
 ['pemerintah', 'dpr', 'israel', 'bppn', 'kota', 'aceh'],
 ['massa', 'rupiah', 'bunga', 'mpr', 'bank', 'suku']]

## Latent Dirichlet Allocation (LDA) 

teknik yang probabilistik

In [22]:
from sklearn.decomposition import LatentDirichletAllocation

In [24]:
lda = LatentDirichletAllocation(n_components=10, max_iter=10, random_state=42)
lda_matrix = lda.fit_transform(bow_matrix)

In [25]:
get_topic(lda)

[['tandjung', 'dpr', 'hukum', 'ketua'],
 ['bank', 'indonesia', 'pemerintah', 'ekonomi'],
 ['pesawat', 'orang', 'rumah', 'korban'],
 ['banjir', 'air', 'warga', 'jakarta', 'jalan'],
 ['tni', 'hukum', 'ginandjar', 'tim'],
 ['rp', 'pemerintah', 'harga', 'bppn'],
 ['indonesia', 'as', 'pemerintah', 'aceh'],
 ['presiden', 'dpr', 'politik', 'ketua'],
 ['daerah', 'indonesia', 'masyarakat', 'maluku'],
 ['polisi', 'kepala', 'jakarta', 'orang']]

unsupervise kita yang menyimpulkan, mesin hanya mencari pola